In [1]:
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
from transformers import Trainer, TrainingArguments

c:\Users\anees\Desktop\Coding\llmrepo\train-LLAMA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
raw_dataset = load_dataset("json", data_files={"train": "dataset_processed.jsonl"})
train_dataset = raw_dataset["train"]

In [7]:
model_path = "D:/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token 

In [8]:
def generate_prompt(example):
    if example["input"]:
        return f"Instruction: {example['instruction']}\nInput: {example['input']}\nResponse:"
    else:
        return f"Instruction: {example['instruction']}\nResponse:"

In [9]:
def tokenize(example):
    prompt = generate_prompt(example)
    target = example["output"]
    full_prompt = prompt + " " + target
    tokenized = tokenizer(
        full_prompt,
        truncation=True,
        max_length=512,  
        padding="max_length"
    )
    prompt_ids = tokenizer(prompt, truncation=True, max_length=512).input_ids
    user_prompt_len = len(prompt_ids)
    labels = tokenized["input_ids"].copy()
    labels[:user_prompt_len] = [-100] * user_prompt_len
    
    tokenized["labels"] = labels

    return tokenized

In [10]:
train_dataset = train_dataset.map(tokenize, batched=False)

Map: 100%|██████████| 82/82 [00:00<00:00, 574.54 examples/s]


In [12]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,        
    torch_dtype=torch.float16,
    device_map="auto"
)
model = model.to("cuda")

In [13]:
lora_config = LoraConfig(
    r=8,                         
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [14]:
model = get_peft_model(model, lora_config)

In [22]:
training_args = TrainingArguments(
    output_dir="./lora-llama-3.2-1b-output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10, 
    logging_dir='./logs',
    logging_strategy="steps",
    evaluation_strategy="no",
    eval_steps=50, 
    save_strategy="epoch",
    save_total_limit=2,
)


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  
    
)

ValueError: You have set `args.eval_strategy` to IntervalStrategy.STEPS but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 

In [17]:
trainer.train()

100%|██████████| 15/15 [01:28<00:00,  5.91s/it]

{'train_runtime': 88.6008, 'train_samples_per_second': 2.776, 'train_steps_per_second': 0.169, 'train_loss': 3.616646575927734, 'epoch': 2.57}


TrainOutput(global_step=15, training_loss=3.616646575927734, metrics={'train_runtime': 88.6008, 'train_samples_per_second': 2.776, 'train_steps_per_second': 0.169, 'total_flos': 634329885573120.0, 'train_loss': 3.616646575927734, 'epoch': 2.571428571428571})

In [18]:
model.save_pretrained("./lora-llama-3.2-1b-adapter")

In [19]:
tokenizer.save_pretrained("./lora-llama-3.2-1b-adapter")

('./lora-llama-3.2-1b-adapter\\tokenizer_config.json',
 './lora-llama-3.2-1b-adapter\\special_tokens_map.json',
 './lora-llama-3.2-1b-adapter\\tokenizer.json')